In [1]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [2]:
!pip install --upgrade transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 23.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 20.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 47.2 MB/s  0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 1.0.0rc2
    Uninstalling huggingface-hub-1.0.0rc2:
      Successfully uninstalled huggingface-hub-1.0.0rc2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [transformers]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependenc

In [3]:
import kagglehub
import torch
import pandas as pd
import numpy as np
import os
import re
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
# from google.colab import drive
from transformers import XLMRobertaTokenizer
from transformers.trainer_utils import get_last_checkpoint
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torch.utils.data import Dataset
from transformers import (
    XLMRobertaForSequenceClassification,
    XLMRobertaTokenizer, 
    Trainer,
    TrainingArguments,
)
from peft import get_peft_model, LoraConfig, TaskType
from transformers.trainer_utils import get_last_checkpoint
from transformers import AutoTokenizer
import wandb
from kaggle_secrets import UserSecretsClient
from peft import PeftModel

2025-10-26 15:19:49.628092: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761491989.867448      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761491989.932818      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `ty

# Datasets
Currently using:
1. https://www.kaggle.com/datasets/infamouscoder/depression-reddit-cleaned
2. https://www.kaggle.com/datasets/rishabhkausish/reddit-depression-dataset

Waiting approval for two primary datasets:
1. https://dcapswoz.ict.usc.edu/daic-woz-database-download/
2. https://ir.cs.georgetown.edu/resources/smhd.html

## Dataset 1

In [4]:
# # Download latest version
# path = kagglehub.dataset_download("infamouscoder/depression-reddit-cleaned")

# print("Path to dataset files:", path)

In [5]:
# csv_files = [f for f in os.listdir(path) if f.endswith('.csv')]

# if csv_files:
#     csv_file_path = os.path.join(path, csv_files[0])
#     df = pd.read_csv(csv_file_path)
#     print("DataFrame head:")
#     display(df.head())
# else:
#     print("No CSV file found in the downloaded directory.")

## Dataset 2

In [6]:
# # Download latest version
# path_2 = kagglehub.dataset_download("rishabhkausish/reddit-depression-dataset")

# print("Path to dataset files:", path)

In [7]:
# csv_files = [f for f in os.listdir(path_2) if f.endswith('.csv')]

# if csv_files:
#     csv_file_path = os.path.join(path_2, csv_files[0])
#     df_2 = pd.read_csv(csv_file_path)
#     print("DataFrame head:")
#     display(df_2.head())
# else:
#     print("No CSV file found in the downloaded directory.")

In [8]:
# df_2 = df_2[['body', 'label']]

In [9]:
# pd.isnull(df).sum(), pd.isnull(df_2).sum()

In [10]:
# df_2 = df_2.dropna()
# print("DataFrame df_2 after removing null rows:")
# display(df_2.head())

## Data Preprocessing

In [11]:
# # Rename columns and combine the two dataframes
# df_renamed = df.rename(columns={'clean_text': 'text', 'is_depression': 'label'})
# df_2_renamed = df_2.rename(columns={'body': 'text', 'label': 'label'})

# combined_df = pd.concat([df_renamed[['text', 'label']], df_2_renamed[['text', 'label']]], ignore_index=True)

# print("Combined DataFrame head:")
# display(combined_df.head())

# print("\nCombined DataFrame info:")
# combined_df.info()

In [12]:
# def count_words(text):
#     """Counts the number of words in a string."""
#     if isinstance(text, str):
#         return len(text.split())
#     return 0

# # Apply word count and filter for rows with less than 10 words
# combined_df['word_count'] = combined_df['text'].apply(count_words)
# short_text_rows = combined_df[combined_df['word_count'] < 10]

# print("Rows with less than 10 words:")
# display(short_text_rows.head())

# print(f"\nNumber of rows with less than 10 words: {len(short_text_rows)}")

In [13]:
# # Remove rows with less than 10 words
# filtered_df = combined_df[combined_df['word_count'] >= 10].copy()

# print("DataFrame after removing rows with less than 10 words:")
# display(filtered_df.head())

# print(f"\nNumber of rows after removing short texts: {len(filtered_df)}")

In [14]:
# drive.mount('/content/drive')
# drive_folder_path = '/content/drive/MyDrive/MH Transfer Learning/'
# file_path = os.path.join(drive_folder_path, 'stopwords-en.txt')

# try:
#     with open(file_path, "r", encoding="utf-8") as f:
#       stopwords_en = [line.strip() for line in f if line.strip()]
# except FileNotFoundError:
#     print(f"Error: The file '{file_path}' was not found. Please ensure the file exists in your Google Drive.")
# except Exception as e:
#     print(f"An error occurred while loading the file: {e}")

In [15]:
# def remove_stopwords(text, stop_words):
#     """Removes stop words from a string."""
#     if isinstance(text, str):
#         words = text.split()
#         filtered_words = [word for word in words if word.lower() not in stop_words]
#         return ' '.join(filtered_words)
#     return ""

# # Apply stop word removal to the 'text' column of filtered_df
# print("Removing stop words from filtered_df...")
# filtered_df['text_no_stopwords'] = filtered_df['text'].apply(lambda x: remove_stopwords(x, set(stopwords_en)))

# print("DataFrame after removing stop words:")
# display(filtered_df.head())

## XLM-RoBERTa-base Model Tokenizer

In [16]:
# # --- Configuration ---
# # This specifies the exact tokenizer that matches the XLM-RoBERTa-base model.
# TOKENIZER_NAME = 'xlm-roberta-base'
# # The maximum sequence length for the model. Texts will be padded or truncated to this length.
# MAX_LENGTH = 256

# def normalize_text(text: str) -> str:
#     """
#     Cleans and normalizes a string of text.

#     This function performs the following steps:
#     1. Converts text to lowercase.
#     2. Removes URLs.
#     3. Removes non-alphanumeric characters (keeps basic punctuation for context).
#     4. Standardizes whitespace by replacing multiple spaces/newlines with a single space.

#     Args:
#         text: The input string to clean.

#     Returns:
#         The cleaned and normalized string.
#     """
#     if not isinstance(text, str):
#         return ""
#     text = text.lower()
#     # Remove URLs
#     text = re.sub(r'https?://\S+|www\.\S+', '', text)
#     # Remove HTML tags
#     text = re.sub(r'<.*?>', '', text)
#     # Keep letters, numbers, and basic punctuation, remove others
#     text = re.sub(r'[^a-zа-я0-9\s.,!?-]', '', text)
#     # Standardize whitespace
#     text = re.sub(r'\s+', ' ', text).strip()
#     return text

# def process_dataframe(df: pd.DataFrame, text_column: str, label_column: str, tokenizer) -> pd.DataFrame:
#     """
#     Cleans, tokenizes, and prepares a DataFrame for model input.

#     Args:
#         df: The input pandas DataFrame.
#         text_column: The name of the column containing the text data.
#         label_column: The name of the column containing the labels.
#         tokenizer: An initialized Hugging Face tokenizer instance.

#     Returns:
#         A new DataFrame with cleaned text, original label, and tokenized input_ids and attention_mask.
#     """
#     if text_column not in df.columns or label_column not in df.columns:
#         print(f"Error: DataFrame must contain '{text_column}' and '{label_column}' columns. Skipping processing.")
#         return pd.DataFrame() # Return empty DataFrame if columns are missing

#     print(f"--- Processing DataFrame with text column '{text_column}' ---")

#     # 1. Normalize Text
#     print("Normalizing text...")
#     df['clean_text'] = df[text_column].apply(normalize_text)

#     # 2. Tokenize Text
#     print("Tokenizing text (this may take a moment)...")
#     encodings = tokenizer(
#         df['clean_text'].tolist(),
#         truncation=True,
#         padding='max_length',
#         max_length=MAX_LENGTH,
#         return_tensors='pt' # Returns PyTorch tensors
#     )

#     # Add tokenized results and original label to a new DataFrame
#     processed_df = pd.DataFrame({
#         'clean_text': df['clean_text'],
#         'label': df[label_column],
#         'input_ids': encodings['input_ids'].tolist(),
#         'attention_mask': encodings['attention_mask'].tolist()
#     })

#     print("Processing complete.")
#     return processed_df


# if __name__ == '__main__':
#     # Initialize the tokenizer
#     print("Initializing XLM-RoBERta tokenizer...")
#     tokenizer = XLMRobertaTokenizer.from_pretrained(TOKENIZER_NAME)

#     processed_df = process_dataframe(filtered_df.copy(), 'text_no_stopwords', 'label', tokenizer)

## Mount Google Drive

In [17]:
# # Mount Google Drive
# drive.mount('/content/drive')

# # Define the path to the desired folder in Google Drive
# drive_folder_path = '/content/drive/MyDrive/MH Transfer Learning/English Depression'

# # Create the folder if it doesn't exist
# if not os.path.exists(drive_folder_path):
#     os.makedirs(drive_folder_path)
#     print(f"Folder '{drive_folder_path}' created in Google Drive.")
# else:
#     print(f"Folder '{drive_folder_path}' already exists in Google Drive.")

# # Define the file path for saving the combined DataFrame
# file_path = os.path.join(drive_folder_path, 'processed_tokenized.csv')

# # Save the combined DataFrame to a CSV file
# processed_df.to_csv(file_path, index=False)

# print(f"Combined DataFrame successfully saved to '{file_path}'")

In [18]:
# # Mount Google Drive (this might already be mounted)
# drive.mount('/content/drive')

# # Define the path to the combined dataset file in Google Drive
# drive_folder_path = '/content/drive/MyDrive/MH Transfer Learning/English Depression'
# file_path = os.path.join(drive_folder_path, 'processed_tokenized.csv')

# # Load the combined DataFrame
# try:
#     loaded_df = pd.read_csv(file_path)
#     print("Combined DataFrame loaded successfully:")
#     display(loaded_df.head())
#     print("\nLoaded DataFrame info:")
#     loaded_df.info()
# except FileNotFoundError:
#     print(f"Error: The file '{file_path}' was not found. Please ensure the file exists in your Google Drive.")
# except Exception as e:
#     print(f"An error occurred while loading the file: {e}")

## Train/Val/Test Split

In [19]:
DATA_FILE_PATH = "/kaggle/input/processed-tokenized/processed_tokenized.csv"
MODEL_NAME = "xlm-roberta-base"
OUTPUT_DIR = "/kaggle/working/xlm-roberta-finetuned/"

# --- Parameters for reproducible data splitting ---
TEST_SPLIT_SIZE = 0.15
VALIDATION_SPLIT_SIZE = 0.15
RANDOM_STATE = 42 # Using a fixed number ensures the same split every time

# --- 2. LOAD AND SPLIT THE DATASET ---
print("✅ Loading and splitting the dataset...")

# Load the full dataset from your private Kaggle source
df = pd.read_csv(DATA_FILE_PATH)

df.dropna(subset=['clean_text'], inplace=True)

# First, split into training and a temporary set (validation + test)
train_df, temp_df = train_test_split(
    df,
    test_size=TEST_SPLIT_SIZE + VALIDATION_SPLIT_SIZE,
    random_state=RANDOM_STATE,
    stratify=df['label'] # Ensures label distribution is the same across splits
)

# Then, split the temporary set into validation and test sets
# Note: The test_size here is calculated relative to the temp_df size
val_df, test_df = train_test_split(
    temp_df,
    test_size=TEST_SPLIT_SIZE / (TEST_SPLIT_SIZE + VALIDATION_SPLIT_SIZE),
    random_state=RANDOM_STATE,
    stratify=temp_df['label']
)

print(f"Data split complete:")
print(f"Training set size: {len(train_df)}")
print(f"Validation set size: {len(val_df)}")
print(f"Test set size: {len(test_df)}")

# --- 3. CREATE PYTORCH DATASET ---
print("\n✅ Preparing PyTorch Datasets...")
tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_NAME)

class TextClassificationDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=256):
        self.tokenizer = tokenizer
        self.texts = dataframe['clean_text'].tolist()
        self.labels = dataframe['label'].tolist()
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

train_dataset = TextClassificationDataset(train_df, tokenizer)
val_dataset = TextClassificationDataset(val_df, tokenizer)
test_dataset = TextClassificationDataset(test_df, tokenizer)

# --- 4. CONFIGURE MODEL AND PEFT (LoRA) ---
print("\n✅ Configuring PEFT (LoRA) model...")
model = XLMRobertaForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=df['label'].nunique())

# LoRA is a form of adapter-tuning that is very memory-efficient
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=['query', 'value']
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

✅ Loading and splitting the dataset...
Data split complete:
Training set size: 1100850
Validation set size: 235896
Test set size: 235897

✅ Preparing PyTorch Datasets...


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]


✅ Configuring PEFT (LoRA) model...


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 887,042 || all params: 278,932,228 || trainable%: 0.3180


## Adapter-Based Fine-tuning


In [20]:
# --- 5. SET TRAINING ARGUMENTS ---
print("\n✅ Setting Training Arguments...")
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=f"{OUTPUT_DIR}/logs",
    logging_steps=1000,
    eval_strategy="steps",
    eval_steps=10000,
    save_strategy="steps",
    save_steps=10000,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    greater_is_better=False,
    fp16=True, # Use mixed precision for faster training on GPUs
    save_total_limit=3 # Only keep the 3 best/most recent checkpoints
)


✅ Setting Training Arguments...


In [21]:
# Retrieve the secret key and login
user_secrets = UserSecretsClient()
wandb_api_key = user_secrets.get_secret("wandb_api_key")
wandb.login(key=wandb_api_key)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: alakbarlic (alakbarlic-ada-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [22]:
# --- 6. INITIALIZE AND START TRAINER ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

print("\n🚀 Starting trainer...")

CHECKPOINT_INPUT_DIR = "/kaggle/input/xlm-roberta-checkpoints/checkpoint-103206"
# Check if a last checkpoint exists in the INPUT directory
last_checkpoint = get_last_checkpoint(CHECKPOINT_INPUT_DIR)


🚀 Starting trainer...


In [23]:
# if last_checkpoint is not None:
#     print(f"✅ Checkpoint found at {last_checkpoint}. Resuming training...")
#     # This resumes from the OLD checkpoint, but saves NEW ones to OUTPUT_DIR
#     trainer.train(resume_from_checkpoint=last_checkpoint)
# else:
#     print("🚀 No checkpoint found. Starting training from scratch...")
#     trainer.train()

# print("\n🎉 Training complete!")

## Train Set Prediction

In [24]:
# Load its state file to find the path to the *best* checkpoint
state_path = os.path.join(last_checkpoint, "trainer_state.json")
with open(state_path, "r") as f:
    state = json.load(f)

# This is the path to the checkpoint that had the best score
# state['best_checkpoint_path'] = /kaggle/working/xlm-roberta-finetuned/checkpoint-100000
best_checkpoint_path = "/kaggle/input/xlm-roberta-checkpoints/checkpoint-100000/checkpoint-100000"
print(f"Loading best model from: {best_checkpoint_path}")

# Load the best adapter weights into your new trainer's model
# (trainer.model currently has an untrained, base adapter)
trainer.model.load_adapter(best_checkpoint_path, adapter_name="default")
print("✅ Best model successfully loaded into the Trainer.")

Loading best model from: /kaggle/input/xlm-roberta-checkpoints/checkpoint-100000/checkpoint-100000
✅ Best model successfully loaded into the Trainer.


In [25]:
prediction_output = trainer.predict(train_dataset)

# prediction_output contains:
# - predictions (the raw logits)
# - label_ids (the true labels)
# - metrics (like 'test_loss', 'test_accuracy', etc.)

print(prediction_output.metrics)

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


wandb: Tracking run with wandb version 0.21.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20251026_181017-wqjj2psu
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run avid-hill-11
wandb: ⭐️ View project at https://wandb.ai/alakbarlic-ada-university/huggingface
wandb: 🚀 View run at https://wandb.ai/alakbarlic-ada-university/huggingface/runs/wqjj2psu


{'test_loss': 0.18963198363780975, 'test_runtime': 10144.2945, 'test_samples_per_second': 108.519, 'test_steps_per_second': 1.696}


In [26]:
def get_classification_report(prediction_output, target_names):
  """
  Generates a classification report from a Hugging Face Trainer's predict() output.

  Args:
    prediction_output: The output object from trainer.predict().
    target_names (list): A list of strings for the class names 
                         (e.g., ['control', 'depression']).
  
  Returns:
    tuple: (report_str, predictions, true_labels)  <-- NOW RETURNS 3 VALUES
  """
  # Get the raw predictions (logits)
  logits = prediction_output.predictions

  # Convert logits to class labels (0 or 1)
  predictions = np.argmax(logits, axis=1)

  # Get the true labels
  true_labels = prediction_output.label_ids

  # Generate a detailed report
  report_str = classification_report(
      true_labels, 
      predictions, 
      target_names=target_names
  )
  
  # Return all three items
  return report_str, predictions, true_labels

In [27]:
report_str, final_predictions, true_labels = get_classification_report(
    prediction_output,
    target_names=['control', 'depression']
)
print(report_str)

              precision    recall  f1-score   support

     control       0.95      0.94      0.95    785063
  depression       0.86      0.88      0.87    315787

    accuracy                           0.93   1100850
   macro avg       0.91      0.91      0.91   1100850
weighted avg       0.93      0.93      0.93   1100850



In [28]:
# --- DEFINE SAVE DIRECTORY ---
PREDICTION_SAVE_DIR = os.path.join(OUTPUT_DIR, "prediction_results_train")
os.makedirs(PREDICTION_SAVE_DIR, exist_ok=True)
print(f"✅ Saving results to: {PREDICTION_SAVE_DIR}")

# --- SAVE THE CLASSIFICATION REPORT (STRING) ---
report_path = os.path.join(PREDICTION_SAVE_DIR, "classification_report.txt")
with open(report_path, "w") as f:
    f.write(report_str)
print(f"Report saved to: {report_path}")

# --- SAVE THE METRICS (DICTIONARY) ---
metrics_path = os.path.join(PREDICTION_SAVE_DIR, "metrics.json")
with open(metrics_path, "w") as f:
    json.dump(prediction_output.metrics, f, indent=4)
print(f"Metrics saved to: {metrics_path}")

# --- SAVE THE PREDICTIONS AND LABELS (ARRAYS) ---
np.save(os.path.join(PREDICTION_SAVE_DIR, "final_predictions.npy"), final_predictions)
np.save(os.path.join(PREDICTION_SAVE_DIR, "true_labels.npy"), true_labels)
np.save(os.path.join(PREDICTION_SAVE_DIR, "raw_logits.npy"), prediction_output.predictions)
print("✅ Predictions, labels, and logits saved as .npy files.")

print("\n🎉 All prediction outputs saved successfully! 🎉")

✅ Saving results to: /kaggle/working/xlm-roberta-finetuned/prediction_results_train
Report saved to: /kaggle/working/xlm-roberta-finetuned/prediction_results_train/classification_report.txt
Metrics saved to: /kaggle/working/xlm-roberta-finetuned/prediction_results_train/metrics.json
✅ Predictions, labels, and logits saved as .npy files.

🎉 All prediction outputs saved successfully! 🎉


## Validation Set Prediction

In [29]:
prediction_output = trainer.predict(val_dataset)

print(prediction_output.metrics)

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'test_loss': 0.19221439957618713, 'test_runtime': 2180.0938, 'test_samples_per_second': 108.205, 'test_steps_per_second': 1.691}


In [30]:
report_str, final_predictions, true_labels = get_classification_report(
    prediction_output,
    target_names=['control', 'depression']
)
print(report_str)

              precision    recall  f1-score   support

     control       0.95      0.94      0.95    168227
  depression       0.86      0.88      0.87     67669

    accuracy                           0.92    235896
   macro avg       0.91      0.91      0.91    235896
weighted avg       0.92      0.92      0.92    235896



In [31]:
PREDICTION_SAVE_DIR = os.path.join(OUTPUT_DIR, "prediction_results_val")
os.makedirs(PREDICTION_SAVE_DIR, exist_ok=True)

report_path = os.path.join(PREDICTION_SAVE_DIR, "classification_report.txt")
with open(report_path, "w") as f:
    f.write(report_str)

metrics_path = os.path.join(PREDICTION_SAVE_DIR, "metrics.json")
with open(metrics_path, "w") as f:
    json.dump(prediction_output.metrics, f, indent=4)

np.save(os.path.join(PREDICTION_SAVE_DIR, "final_predictions.npy"), final_predictions)
np.save(os.path.join(PREDICTION_SAVE_DIR, "true_labels.npy"), true_labels)
np.save(os.path.join(PREDICTION_SAVE_DIR, "raw_logits.npy"), prediction_output.predictions)

## Test Set Prediction

In [32]:
prediction_output = trainer.predict(test_dataset)


print(prediction_output.metrics)

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'test_loss': 0.19124729931354523, 'test_runtime': 2180.1696, 'test_samples_per_second': 108.201, 'test_steps_per_second': 1.691}


In [33]:
report_str, final_predictions, true_labels = get_classification_report(
    prediction_output,
    target_names=['control', 'depression']
)
print(report_str)

              precision    recall  f1-score   support

     control       0.95      0.94      0.95    168228
  depression       0.86      0.88      0.87     67669

    accuracy                           0.93    235897
   macro avg       0.91      0.91      0.91    235897
weighted avg       0.93      0.93      0.93    235897



In [34]:
PREDICTION_SAVE_DIR = os.path.join(OUTPUT_DIR, "prediction_results_test")
os.makedirs(PREDICTION_SAVE_DIR, exist_ok=True)

report_path = os.path.join(PREDICTION_SAVE_DIR, "classification_report.txt")
with open(report_path, "w") as f:
    f.write(report_str)

metrics_path = os.path.join(PREDICTION_SAVE_DIR, "metrics.json")
with open(metrics_path, "w") as f:
    json.dump(prediction_output.metrics, f, indent=4)

np.save(os.path.join(PREDICTION_SAVE_DIR, "final_predictions.npy"), final_predictions)
np.save(os.path.join(PREDICTION_SAVE_DIR, "true_labels.npy"), true_labels)
np.save(os.path.join(PREDICTION_SAVE_DIR, "raw_logits.npy"), prediction_output.predictions)